## Отчет по исследованию Looped Transformers и их возможностей.
Мини исследование-отчет по статье - https://arxiv.org/abs/2311.12424

### Сравнение эффективности Looped TFs с обычными TFs
Все эксперементы проводились с выключенным Mixed Precision, по скольку он не влиял на скорость обучения - видимо из-за того что gpt2_nano не может использовать GPU эффективно.
Одна часть эксперементов проводилась на локальной машине с Nvidia 3060ti mobile, а вторая на удаленной машине с Nvidia Tesla P100.

Посмотрим на скорость сходимости трансформера (TF) с параметром $ L = 12 $ и Looped TFs с параметрами $ L = 1; b = \{5, 10, 20, 25\} T= \{10, 20\}$. Были использованы следующие параметры Heads=4, dims=10, points=31. Потери масштабированы на размерность регрессии.

|                                                    |                                               |
|----------------------------------------------------|-----------------------------------------------|
| ![](../images/Compare_losses_b_n_t_10.png "Title") | ![](../images/Compare_losses_b_n.png "Title") |

Как можно видеть из графиков, Looped TFs при увеличении $ b $ дает улучшение метрик. При количестве параметров в 12 раз меньшим, чем у обычного трансформера, он показывает неплохие показатели. Конфигурации для запуска моделей взяты из папки эксперимента №5.

### Итеративные свойства
По скольку в статье подразумевается что Looped TFs имеют итеративные свойства, мы можем их проверить на примере линейной регрессии.
Для этого обучим Looped TFs со значениями $ b=\{5, 10, 20\}; T=\{ 20, 10\} $  и посмотрим на их сходимость при больших $ b $.

|           |       |
|-----------|-------|
| ![alt text](../images/check_for_convergence_properties.png "Title")    | ![alt text](../images/check_for_scheduling_convergence_properties_noisy_linear_regression_T_10_short.png "Title") |

Из графика можем видеть что при большем $ b $ на тренировке Looped TF показывает себя лучше при $ b > 5 $.  Блокнот `experiment_schedule.ipynb`. 


### Дополнительно
Появилась гипотеза о том, что шедулинг может помочь для обобщающих способностей модели. Увеличивая параметр $ b $ во время тренировки можно получить лучшую обобщающую способность, однако в процессе эксперимента со слабыми моделями это явно не подтвердилось. При тренировке с вариативным & b & и , модель дает меньшую ошибку на большем количестве итераций при $ T=20 $, однако при $ T=10 $ утверждать то же не можем.  Опыты проводились в блокноте `experiment_shedule.ipynb` и в блокноте `experiment_shedule_2.ipynb`.

|           |                                                                                              |
|-----------|----------------------------------------------------------------------------------------------|
| ![](../images/check_for_scheduling_convergence_properties.png "Title")    | ![](../images/check_for_scheduling_convergence_properties_train_steps_10000.png "Title")     |





Из графика можно видеть, что в отличии от моделей с параметрами $ b=5; b=10; b=15 $, у модели с варьированием $ b $ на этапе тренировки показатели точности сильно разнятся. 

### Попробуем брать только N последних токенов
Проверим гипотезу

> ***Гипотеза***
> 
> В таком случае, модель имеет возможность использовать часть токенов как хранилище информации с предыдущего шага.

Чтобы понять на сколько мы можем обрезать количество токенов которые подаются модель замаскируем часть из них и посмотрим на метрики.
Модели обучались с параметрами b = \{5, 10, 20\} и T = \{10, 20\} при количестве шагов равным 20000. 

На графиках ниже мы можем видеть вполне ожидаемый результат, для моделей с $ b = \{5, 10, 20\} $ обученных с параметром $ T = \{10, 20\}$ точность ухудшается вместе с уменьшением количества токенов поступающих на вход. 

![](../images/check_last_n_tokens_quality.png "check_last_n_tokens_quality") 

Также интересно, что при увеличении количества итераций при инференсе мы получаем такую же ошибку на каждом этапе отбрасывания $ n $ токенов. 
Можно сделать вывод что использование последних $ n $ токенов сильно влияет на качество, но что если брать $ n $ первых токенов. Попробуем маскировать все токены кроме $ n $ первых и посмотрим на метрики модели.

![](../images/check_first_n_tokens_quality.png "check_first_n_tokens_quality") 

Как можем видеть ощутимой разницы это не дало, видимо для зацикленных трансформеров первые и последние токены имеют схожую важность. Меньшее количество токенов негативно влияет на итоговые показатели модели. Эксперименты проводились в блокнотах `experiment_last_n_tokens.ipynb` и `experiment_first_n_tokens.ipynb`.

## Looped n-layers.
Проверим возможности зацикленных трансформеров при обучении с количеством слоев $ L > 1 $. 